In [ ]:
import pandas as pd


def generate_team_strength():

    # Load CSVs
    club_games = pd.read_csv("../data/input/club_games.csv")
    clubs = pd.read_csv("../data/input/clubs.csv")
    games = pd.read_csv("../data/input/games.csv")
    competitions = pd.read_csv("../data/input/competitions.csv")

    # Merge club names and game metadata
    club_games = club_games.merge(clubs, left_on="club_id", right_on="club_id", suffixes=("", "_club"))
    club_games = club_games.merge(games, left_on="game_id", right_on="game_id", suffixes=("", "_game"))
    club_games = club_games.merge(competitions, left_on="competition_id", right_on="competition_id", suffixes=("", "_comp"))

    club_games = club_games[club_games["name_comp"] == "premier-league"]

    team_stats = (
        club_games.groupby(["name", "season"])
        .agg(
            games_played=("is_win", "count"),
            wins=("is_win", "sum"),
            goals_scored=("own_goals", "sum"),
            goals_conceded=("opponent_goals", "sum"),
            #squad_size=("squad_size", "mean"),
            #avg_age=("average_age", "mean"),
            #market_value=("total_market_value", "mean"),
        )
        .reset_index()
    )

    team_stats["draws"] = team_stats["games_played"] - team_stats["wins"] - (
        club_games.groupby(["name", "season"])["is_win"].apply(lambda x: (~x & (x != None)).sum()).values
    )

    team_stats["losses"] = team_stats["games_played"] - team_stats["wins"] - team_stats["draws"]
    team_stats["points"] = 3 * team_stats["wins"] + 1 * team_stats["draws"]
    team_stats["ppg"] = team_stats["points"] / team_stats["games_played"]
    team_stats["goal_difference"] = team_stats["goals_scored"] - team_stats["goals_conceded"]

    return team_stats



In [42]:
top_teams

,name,season,games_played,wins,goals_scored,goals_conceded,squad_size,avg_age,market_value,draws,losses,points,ppg,goal_difference
124,Liverpool Football Club,2019,38,32,85.0,33.0,25.0,26.6,NaN,0,6,96,2.526316,52.0
136,Manchester City Football Club,2017,38,32,106.0,27.0,28.0,26.7,NaN,0,6,96,2.526316,79.0
137,Manchester City Football Club,2018,38,32,95.0,23.0,28.0,26.7,NaN,0,6,96,2.526316,72.0
123,Liverpool Football Club,2018,38,30,89.0,22.0,25.0,26.6,NaN,0,8,90,2.368421,67.0
57,Chelsea Football Club,2016,38,30,85.0,33.0,30.0,23.7,NaN,0,8,90,2.368421,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Queens Park Rangers,2012,38,4,30.0,60.0,26.0,25.9,NaN,0,34,12,0.315789,-30.0
183,Sheffield United,2023,38,3,35.0,104.0,28.0,24.2,NaN,0,35,9,0.236842,-69.0
24,Aston Villa Football Club,2015,38,3,27.0,76.0,24.0,27.7,NaN,0,35,9,0.236842,-49.0
99,Huddersfield Town,2018,38,3,22.0,76.0,33.0,26.7,NaN,0,35,9,0.236842,-54.0
